Samuel Grimm,
March and April 2023

# Summary

Public transport companies in Switzerland provide live data about their vehicle like where they arrived on which time and what time was planned.

In this project, we are exploring this data.

SBB provides it's data live. (Almost) always when a train enters (arrival) and leaves (departure) a railway station, this data including the expected data (that you can also find in the app by SBB) is sent and collected in the "ist-daten" dataset. SBB provides also an dataset with all that data from the last day via an API where you can also download the data as CSV:

Dataset: [Soll/Ist Vergleich Abfahrts-/Ankunftszeiten SBB](https://data.sbb.ch/explore/dataset/ist-daten-sbb/table/?flg=de&dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJjb2x1bW4iLCJmdW5jIjoiQ09VTlQiLCJ5QXhpcyI6Imxpbmllbl9pZCIsInNjaWVudGlmaWNEaXNwbGF5Ijp0cnVlLCJjb2xvciI6InJhbmdlLUFjY2VudCJ9XSwieEF4aXMiOiJ2ZXJrZWhyc21pdHRlbF90ZXh0IiwibWF4cG9pbnRzIjoyMCwidGltZXNjYWxlIjoiIiwic29ydCI6IiIsInNlcmllc0JyZWFrZG93biI6ImFua3VuZnRzdmVyc3BhdHVuZyIsInN0YWNrZWQiOiJwZXJjZW50IiwiY29uZmlnIjp7ImRhdGFzZXQiOiJpc3QtZGF0ZW4tc2JiIiwib3B0aW9ucyI6eyJmbGciOiJkZSJ9fX1dLCJ0aW1lc2NhbGUiOiIiLCJkaXNwbGF5TGVnZW5kIjp0cnVlLCJhbGlnbk1vbnRoIjp0cnVlfQ%3D%3D)

But sure, this is only a subset of the whole picture in Switzerland. There are other companies than SBB as well! We can also travel back in time. To do so, we can use Opentransportdata's data.

Here are some some older dataset with the same kind of data: https://opentransportdata.swiss/de/dataset/istdaten

And here are the files stored as zip file for the years 2016 - 2023 (2016 and 2017 are incomplete): https://opentransportdata.swiss/de/ist-daten-archiv/

# Imports

In [ ]:
# 1. import of numpy and pandas:
import numpy as np
import pandas as pd

# Show all of our columns in this Jupyter notebook:
pd.set_option('display.max_columns', None)

We define variables holding the column names in order to prevent hard-coded column names within the code and to be able ro rename those if we want to:

In [ ]:
CSV_FILE_PATH = '../data/sbb_data/2023-03-21_ist-daten-sbb.csv'

COLUMN_LOCATION = 'Haltestellen Name'

COLUMN_ARRIVAL_EXPECTED = 'Ankunftszeit'
COLUMN_ARRIVAL_ACTUAL = 'An Prognose'
COLUMN_DEPARTURE_EXPECTED = 'Abfahrtszeit'
COLUMN_DEPARTURE_ACTUAL = 'Ab Prognose'

COLUMN_DELAY_ARRIVAL = 'delay_arrival'
COLUMN_DELAY_DEPARTURE = 'delay_departure'
COLUMN_EARLY_ARRIVAL = 'early_arrival'
COLUMN_EARLY_DEPARTURE = 'early_departure'
COLUMN_DELAY_IN_MIN = 'delay_min'

COLUMN_VEHICLE_TYPE = 'Verkehrsmittel Text'
COLUMN_JOURNEY_ID = "Fahrt Bezeichner"



# Data acquisition
Here, we are loading the csv file and automatically parse the columns with a date to a datetime object.

In [ ]:
date_format = "%Y-%m-%d %H:%M:%S"


COLUMNS_WITH_DATE = [COLUMN_ARRIVAL_EXPECTED,
                     COLUMN_ARRIVAL_ACTUAL,
                     COLUMN_DEPARTURE_EXPECTED,
                     COLUMN_DEPARTURE_ACTUAL]

df = pd.read_csv(CSV_FILE_PATH,
                 sep=';', parse_dates=COLUMNS_WITH_DATE, date_parser=lambda x: pd.to_datetime(x, format=date_format),)


# EDI
(exploratory data analysis)

In [ ]:
df.shape

As we can see, we have 25 columns and 63065 rows.

In [ ]:
df.info()

Accordingly to `info`, "Umlauf ID" is the only column with no valid entries.

The dtypes are in most cases `object`s instead of strings.

The following columns have missing values:
* Produkt ID
* Umlauf ID
* Ankunftszeit
* An Prognose
* An Prognose Status
* Abfahrtszeit
* Ab Prognose
* Ab Prognsoe Status
* Geoposition
* lod

In [ ]:
df.dtypes

## Nominal and Ordinal data:
Here are some examples for Nominal and Ordinal data.

In the following output we see that
* Vor the "Verkehrsmittel Text" we have 14 different categories or companies. The most seems to be an "S" like "S-Bahn". But these categories aren't that consistent.
* we have a lot of different dates for Abfahrszeit (1301) and Ankunftszeit (1301) -> A day has 1’440 minutes. We see that for the most of the minutes of the day, we have entries. But compared to "An Prognose", we have much less because this data seems to be accurate in seconds.

In [ ]:
# nominal data:
df[[COLUMN_VEHICLE_TYPE]].value_counts()
# df[['Verkehrsmittel Text']].value_counts()

# # ordinal data:
# df[['Abfahrtszeit']].value_counts()
# df[['Ankunftszeit']].value_counts()
# df[['An Prognose']].value_counts()

# We'd get discrete date if we add a column "Verspätung Ankunft" (delay) by subtracting "Ankunft" of "An Prognose".


In [ ]:
df.sample(10)

# Data cleansing & data transformation

First, let's rename the columns to something shorter:

In [ ]:
df = df.rename(columns={
    COLUMN_ARRIVAL_EXPECTED: (COLUMN_ARRIVAL_EXPECTED:='arrival_expected'),
    COLUMN_ARRIVAL_ACTUAL: (COLUMN_ARRIVAL_ACTUAL:='arrival_actual'),
    COLUMN_DEPARTURE_EXPECTED: (COLUMN_DEPARTURE_EXPECTED:='departure_expected'),
    COLUMN_DEPARTURE_ACTUAL: (COLUMN_DEPARTURE_ACTUAL:='departure_actual'),
    COLUMN_DELAY_ARRIVAL: (COLUMN_DELAY_ARRIVAL:='delay_arrival'),
    COLUMN_DELAY_DEPARTURE: (COLUMN_DELAY_DEPARTURE:='delay_departure'),
})


Then convert some nominal columns to categorical ones in order to use less memory and faster queries (proposal from Nic :))

In [ ]:
df['Betreiber Name'] = df['Betreiber Name'].astype('category')
df[COLUMN_VEHICLE_TYPE] = df[COLUMN_VEHICLE_TYPE].astype('category')
df['Linien Text'] = df['Linien Text'].astype('category')
df[COLUMN_LOCATION] = df[COLUMN_LOCATION].astype('category')
df['Produkt ID'] = df['Produkt ID'].astype('category')


Sorting the values by their journey id. This is not used later, but this is useful to see that the same train have often the same delay on the same journey.

In [ ]:
df.sort_values(by=[COLUMN_JOURNEY_ID, COLUMN_DEPARTURE_EXPECTED, COLUMN_ARRIVAL_EXPECTED], inplace=True)

df.head(10)

We are first creating the columns delay_arrival and delay_departure in order to be able to do some calculation on the delays.

In [ ]:
def create_column_for_arrival_and_departure(
        column_arrival_actual,
        column_arrival_expected,
        column_departure_expected,
        column_departure_actual,
        new_column_name_arrival_diff,
        new_column_name_departure_diff):

    # Calculate the delays for arrival and and departure:
    df[new_column_name_arrival_diff] = (
        df[column_arrival_actual] - df[column_arrival_expected]).fillna(pd.Timedelta(seconds=0))
    df[new_column_name_departure_diff] = (
        df[column_departure_actual] - df[column_departure_expected]).fillna(pd.Timedelta(seconds=0))

    # Since the times real times are not more precise than 1 minute, we want to only have a delay
    # when the System registered that it is delayed:
    DEFAULT_TIME = pd.Timedelta(hours=0)
    df.loc[df[new_column_name_arrival_diff] < pd.Timedelta(
        minutes=1), new_column_name_arrival_diff] = DEFAULT_TIME
    df.loc[df[new_column_name_departure_diff] < pd.Timedelta(
        minutes=1), new_column_name_departure_diff] = DEFAULT_TIME



create_column_for_arrival_and_departure(column_arrival_expected=COLUMN_ARRIVAL_EXPECTED,
                                        column_arrival_actual=COLUMN_ARRIVAL_ACTUAL,
                                        column_departure_expected=COLUMN_DEPARTURE_EXPECTED,
                                        column_departure_actual=COLUMN_DEPARTURE_ACTUAL,
                                        new_column_name_arrival_diff=COLUMN_DELAY_ARRIVAL,
                                        new_column_name_departure_diff=COLUMN_DELAY_DEPARTURE)



### Which trains have the most delay on their arrival?

In [ ]:
df.loc[df[COLUMN_DELAY_ARRIVAL] > pd.Timedelta(
    minutes=0)].sort_values([COLUMN_DELAY_ARRIVAL], ascending=False)[[
        COLUMN_JOURNEY_ID,
        COLUMN_LOCATION,
        COLUMN_ARRIVAL_EXPECTED,
        COLUMN_ARRIVAL_ACTUAL,
        COLUMN_DEPARTURE_EXPECTED,
        COLUMN_DEPARTURE_ACTUAL,
        COLUMN_DELAY_ARRIVAL,
        COLUMN_DELAY_DEPARTURE
    ]].head(10)


And something similar if the vehicle arrives or departures ahead of schedule.

In [ ]:
#-------
create_column_for_arrival_and_departure(column_arrival_expected=COLUMN_ARRIVAL_ACTUAL,      # Swap actual an expected
                                        column_arrival_actual=COLUMN_ARRIVAL_EXPECTED,      # because we want to know
                                        column_departure_expected=COLUMN_DEPARTURE_ACTUAL,  # the opposite of delay: early.
                                        column_departure_actual=COLUMN_DEPARTURE_EXPECTED,
                                        new_column_name_arrival_diff=COLUMN_EARLY_ARRIVAL,
                                        new_column_name_departure_diff=COLUMN_EARLY_DEPARTURE)

df.loc[df[COLUMN_EARLY_DEPARTURE] > pd.Timedelta(
    minutes=0)].sort_values([COLUMN_EARLY_DEPARTURE], ascending=False).head(10)


### How many trains are too departing too early?

In [ ]:
# How many trains are departing too early?
df.loc[df[COLUMN_EARLY_DEPARTURE] > pd.Timedelta(
    minutes=0)].shape[0]

In [ ]:
### Convert the delay into a digit (float)
#   Later, we need the delay in minutes for the plots.

df[COLUMN_DELAY_IN_MIN] = (df[COLUMN_DELAY_ARRIVAL].dt.total_seconds() / 60).round()

# Visualization

In [ ]:
# Import the libraries for visualization:
import matplotlib.pyplot as plt

A possible data visualization could be (Outdated because lack of time):

* A map where one can see on which location (we have the GPS coordinates) we measured the most delay (we only concentrate on the high delays).
* Which company (like SBB) has how many minutes delays per day in average? (Therefore, we'd use multiple CSVs for multiple days)

### The distribution of delay on arrival

In the following histogram, you can see the distribution of delay on arrival:

In [ ]:
# Plot a histogram of the delay distribution:

plt.hist(df['delay_min'], bins=50)
plt.xlabel('Delay (minutes)')
plt.ylabel('Frequency')

# Zoom in so that we only look at the delays lower than 20 minutes:
plt.xlim(0, 20)
plt.xticks(range(0, 20, 1))
plt.ylim(0, 4_000)

plt.show()

In [ ]:
# Create a box plot of the delay_min column
plt.boxplot(df['delay_min'])

# Add a title and labels to the plot
plt.title('Box Plot of Delay Distribution')
plt.xlabel('Delay (min)')

plt.ylim(0, 20)

# Show the plot
plt.show()

#### Some statistics about the distribution

In [ ]:
print('mean          ', round(df[COLUMN_DELAY_IN_MIN].mean(), 2))
print('median        ', df[COLUMN_DELAY_IN_MIN].median())
print('min           ', df[COLUMN_DELAY_IN_MIN].min())
print('max           ', df[COLUMN_DELAY_IN_MIN].max())
print('top 75%       ', df[COLUMN_DELAY_IN_MIN].quantile(0.75))
print('lowest 25%    ', df[COLUMN_DELAY_IN_MIN].quantile(0.25))

In [ ]:
delay_counts = df.groupby('delay_min').agg({'delay_min': 'count'})

delay_counts

In [ ]:
delays = delay_counts.index.values
counts = delay_counts['delay_min'].values
amount_of_rows = df.shape[0]

print('No delay:', counts[0], f'({round((counts[0] / amount_of_rows) * 100, 1)}%))')
for i in range(1, len(delays)):
    print(f'{delays[i]} min delay:', counts[i], f'({round((counts[i] / amount_of_rows) * 100, 2)}%))')

In [ ]:
print('This means that how many trains have less than (or equals to) 2 minutes of delay?')

count_less_than_2_min = counts[0] + counts[1] + counts[2]
print('2 or less min delay:', count_less_than_2_min, f'of {amount_of_rows}', f'({round((count_less_than_2_min / amount_of_rows) * 100, 2)}%))')

count_more_than_5_min = sum(counts[5:])
print('more than 5 min delay:', count_more_than_5_min, f'of {amount_of_rows}', f'({round((count_more_than_5_min / amount_of_rows) * 100, 2)}%))')

### Which locations had to experience the most delays (max) on this day?

In [ ]:

def subplot(label_name, label_value, title, series):


    fig, ax = plt.subplots(figsize=(8, 6))
    ax.bar(series.index, series.values)
    ax.set_ylabel(label_value)
    ax.set_xlabel(label_name)
    ax.set_xticklabels(series.index, rotation=90)
    ax.set_title(title)
    plt.show()


series = df.groupby(COLUMN_LOCATION)[COLUMN_DELAY_IN_MIN].max()

# Sort the locations by average delay in descending order and select top 10
series = series.sort_values(ascending=True).nlargest(20)
subplot(COLUMN_LOCATION, 'Max delay in arrival', title='Max delay in arrival', series=series)

### Which locations had to experience the most delays on this day **in median**?

In [ ]:
series = df.groupby(COLUMN_LOCATION)[COLUMN_DELAY_IN_MIN].median()

# Sort the locations by average delay in descending order and select top 10
series = series.sort_values(ascending=False).nlargest(20)
subplot(COLUMN_LOCATION, 'Median delay in arrival', title='Median delay in arrival', series=series)

# Data storage (optional)

We can export the calculated delays:

In [ ]:
df_export = df[[
    COLUMN_JOURNEY_ID,
    COLUMN_LOCATION,
    COLUMN_ARRIVAL_EXPECTED,
    COLUMN_ARRIVAL_ACTUAL,
    COLUMN_DEPARTURE_EXPECTED,
    COLUMN_DEPARTURE_ACTUAL,
    COLUMN_DELAY_ARRIVAL,
    COLUMN_DELAY_DEPARTURE,
    COLUMN_EARLY_ARRIVAL,
    COLUMN_EARLY_DEPARTURE,]]

df_export.to_excel('./delay_export.xlsx', index=False)


# References

Dataset: [Soll/Ist Vergleich Abfahrts-/Ankunftszeiten SBB](https://data.sbb.ch/explore/dataset/ist-daten-sbb/table/?flg=de&dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJjb2x1bW4iLCJmdW5jIjoiQ09VTlQiLCJ5QXhpcyI6Imxpbmllbl9pZCIsInNjaWVudGlmaWNEaXNwbGF5Ijp0cnVlLCJjb2xvciI6InJhbmdlLUFjY2VudCJ9XSwieEF4aXMiOiJ2ZXJrZWhyc21pdHRlbF90ZXh0IiwibWF4cG9pbnRzIjoyMCwidGltZXNjYWxlIjoiIiwic29ydCI6IiIsInNlcmllc0JyZWFrZG93biI6ImFua3VuZnRzdmVyc3BhdHVuZyIsInN0YWNrZWQiOiJwZXJjZW50IiwiY29uZmlnIjp7ImRhdGFzZXQiOiJpc3QtZGF0ZW4tc2JiIiwib3B0aW9ucyI6eyJmbGciOiJkZSJ9fX1dLCJ0aW1lc2NhbGUiOiIiLCJkaXNwbGF5TGVnZW5kIjp0cnVlLCJhbGlnbk1vbnRoIjp0cnVlfQ%3D%3D)

Older dataset with the same kind of data: https://opentransportdata.swiss/de/dataset/istdaten

Files stored as zip file for the years 2016 - 2023 (2016 and 2017 are incomplete): https://opentransportdata.swiss/de/ist-daten-archiv/